In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from importlib import reload
import os
import numpy as np
import json
import seaborn as sns

In [ ]:
avg_feat_path = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\results\\Avg_Features'

In [ ]:
# Initialize an empty list to store the DataFrames
dfs = []

# Iterate through the files in the directory
for filename in os.listdir(avg_feat_path):
    if filename.endswith(".json"):
        # Construct the full file path
        file_path = os.path.join(avg_feat_path, filename)
        f = open(file_path)
        data1 = json.load(f)
        df = pd.read_json(data1, orient ='index')
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'FollowUp'}, inplace=True)

        df['SubID'] = [filename.split('_')[0]]*df.shape[0]
        dfs.append(df)


All_AvgFeatures = pd.concat(dfs, ignore_index=True)

In [ ]:
#All_AvgFeatures.to_excel(os.path.join(avg_feat_path,'All_AvgFeatures.xlsx'), header = True, index = False)

In [ ]:
# Filter out rows with FollowUp as 'Beelitz'
beelitz_rows = All_AvgFeatures[All_AvgFeatures['FollowUp'] == 'Beelitz']

# Group by 'SubID' and apply custom aggregation
agg_funcs = {
    'Telemetry Duration Sum': 'sum',
    'Sensing Duration Sum': 'sum',
    'Last Battery Value': 'first',  # Keep only the first value, which will be 'Beelitz'
}

result_df = beelitz_rows.groupby(['SubID', 'FollowUp']).agg(agg_funcs).reset_index()

result_df

In [ ]:
result_df.to_excel(os.path.join(avg_feat_path,'Results_df.xlsx'), header = True, index = True)

In [ ]:
All_AvgFeatures_MrgFU0M = pd.read_excel(os.path.join(
    avg_feat_path, 'All_AvgFeatures_MergedFU0M.xlsx'
))
All_AvgFeatures_MrgFU0M['Telemetry Duration SumHours'] = All_AvgFeatures_MrgFU0M['Telemetry Duration Sum'] / (60 * 60)
All_AvgFeatures_MrgFU0M['TelemetryProgrammingOnlyHours'] = (All_AvgFeatures_MrgFU0M['Telemetry Duration Sum'] - All_AvgFeatures_MrgFU0M['Sensing Duration Sum']) / (60 * 60)

All_AvgFeatures_MrgFU0M.head()

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x='FollowUp', y='TelemetryProgrammingOnlyHours', boxprops=dict(alpha=.8), data=All_AvgFeatures_MrgFU0M, width=0.3, color = 'darkred')
sns.stripplot(data=All_AvgFeatures_MrgFU0M, x='FollowUp', y='TelemetryProgrammingOnlyHours', hue = 'SenSight',jitter = 0, size = 8, palette = 'pastel')

sns.lineplot(
    data=All_AvgFeatures_MrgFU0M, x='FollowUp', y='TelemetryProgrammingOnlyHours', units = 'SubID',
    color=".7", alpha = 0.4, estimator=None, 
    linestyle=':'
)


In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x='FollowUp', y='Last Battery Value', boxprops=dict(alpha=.8), data=All_AvgFeatures_MrgFU0M, width=0.3, color = 'darkred')
sns.stripplot(data=All_AvgFeatures_MrgFU0M, x='FollowUp', y='Last Battery Value', hue = 'SenSight',jitter = 1, size = 8, palette = 'pastel', alpha = 0.7)

sns.lineplot(
    data=All_AvgFeatures_MrgFU0M, x='FollowUp', y='Last Battery Value', units = 'SubID',
    color=".7", alpha = 0.4, estimator=None, 
    linestyle=':'
)

In [ ]:
m3fu_sensing = All_AvgFeatures_MrgFU0M[All_AvgFeatures_MrgFU0M['FollowUp'] == 'FU3M']
plt.figure(figsize=(6,4))
sns.boxplot(x='SenSight', y='TelemetryProgrammingOnlyHours', boxprops=dict(alpha=.8), data=m3fu_sensing, width=0.3, color = 'midnightblue')
sns.stripplot(data=m3fu_sensing, x='SenSight', y='TelemetryProgrammingOnlyHours',jitter = 1, size = 8, palette = 'pastel', alpha = 0.7)

In [ ]:
m3fu_sensing[m3fu_sensing['SenSight'] == 0]

In [ ]:
from scipy import stats
stats.permutation_test((m3fu_sensing))